In [20]:
import os
import pandas as pd

In [21]:
# Set the path to the folder containing your CSV files
folder_path = R"C:\Users\andre\OneDrive - Alma Mater Studiorum Università di Bologna\University\UniBo\Machine Learning\90477_mlds_iii_pr2.20\data"

# List all files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv") if file != 'InfoComune.csv']

# Create an empty dictionary to store DataFrames
dataframes = {}

# Iterate through each CSV file
for file in csv_files:
    # Extract the file name (excluding .csv)
    df_name = os.path.splitext(file)[0]
    
    # Create the DataFrame and store it in the dictionary
    dataframes[df_name] = pd.read_csv(os.path.join(folder_path, file), header=[0,1])

info_path = os.path.join(folder_path, 'InfoComune.csv')

dataframes['info'] = pd.read_csv(info_path, encoding='latin1')


In [22]:
dataframes['bologna']

DATE ---     PROV  ---           MET   ---               ...   SO2  \
      YYYY  MM  DD  COM   GP  A[km2]    TG    TN    TX     HU  ...    Av   
0     2017   1   1   46  194  2586.3  0.34 -3.44  5.83  82.72  ...   ---   
1     2017   1   2   46  194  2586.3  2.27 -2.24  8.13  84.16  ...   ---   
2     2017   1   3   46  194  2586.3  1.92 -1.84  6.88  83.48  ...   ---   
3     2017   1   4   46  194  2586.3  2.82 -2.61  9.56  79.68  ...   ---   
4     2017   1   5   46  194  2586.3  3.01 -3.44  7.99  51.05  ...   ---   
...    ...  ..  ..  ...  ...     ...   ...   ...   ...    ...  ...   ...   
1456  2020  12  27   46  194  2586.3  1.83 -0.08  4.24  60.21  ...  1.25   
1457  2020  12  28   46  194  2586.3  2.22 -0.17  3.23  77.08  ...  0.84   
1458  2020  12  29   46  194  2586.3  1.34 -2.66  3.25  73.42  ...   ---   
1459  2020  12  30   46  194  2586.3  2.93  0.61  5.29  75.44  ...   ---   
1460  2020  12  31   46  194  2586.3  0.94 -3.58  3.04  65.92  ...   ---   

        125  ug.m-3   24h     ---                 >TS       ---         
     Median StDev.5 Max.2 COMx.11 Min.11 COMn.11 GP.9 A[km2].10 A[%].9  
0       ---     ---   ---     ---    ---     ---  ---       ---    ---  
1       ---     ---   ---     ---    ---     ---  ---       ---    ---  
2       ---     ---   ---     ---    ---     ---  ---       ---    ---  
3       ---     ---   ---     ---    ---     ---  ---       ---    ---  
4       ---     ---   ---     ---    ---     ---  ---       ---    ---  
...     ...     ...   ...     ...    ...     ...  ...       ...    ...  
1456   1.24     0.1  1.78    3794   0.77    3809    0        0.     0.  
1457   0.84    0.09  1.37    3794   0.51    3809    0        0.     0.  
1458    ---     ---   ---     ---    ---     ---  ---       ---    ---  
1459    ---     ---   ---     ---    ---     ---  ---       ---    ---  
1460    ---     ---   ---     ---    ---     ---  ---       ---    ---  

[1461 rows x 123 columns]

In [23]:
dataframes['info']

,ComID,CodRIP,CodREG,CodPROV,CodCOM,CodUTS,Name,Area (m2),UTM X (m E),UTM Y (m N),Lat (Deg N),Lon (Deg E),EOBS Points
0,1,1,1,1,201,1001,Agliè,13300000.0,404145.0229,5020000.0,45.365908,7.775986,4
1,2,1,1,1,201,1002,Airasca,16000000.0,380314.3611,4980000.0,44.921764,7.483505,4
2,3,1,1,1,201,1003,Ala di Stura,45800000.0,365323.4970,5020000.0,45.306805,7.282044,4
3,4,1,1,1,201,1004,Albiano d'Ivrea,11000000.0,417371.7088,5030000.0,45.431590,7.943660,4
4,5,1,1,1,201,1006,Almese,17700000.0,375740.6579,5000000.0,45.118005,7.420164,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7898,7899,5,20,111,111,111103,Villaputzu,181000000.0,547384.2633,4380000.0,39.534590,9.551386,7
7899,7900,5,20,111,111,111104,Villasalto,130000000.0,533902.3803,4370000.0,39.468069,9.394129,5
7900,7901,5,20,111,111,111105,Villasimius,57600000.0,544107.7439,4330000.0,39.155338,9.510493,4
7901,7902,5,20,111,111,111106,Villasor,86400000.0,492309.7840,4360000.0,39.383373,8.910706,4


In [24]:
for df in dataframes:
    # change missing values to the proper format
    dataframes[df].replace('---', pd.NA, inplace = True)
    # ensure a unique format
    dataframes[df] = dataframes[df].convert_dtypes()

In [25]:
dataframes['bologna']

DATE ---     PROV  ---           MET   ---               ...   SO2  \
      YYYY  MM  DD  COM   GP  A[km2]    TG    TN    TX     HU  ...    Av   
0     2017   1   1   46  194  2586.3  0.34 -3.44  5.83  82.72  ...  <NA>   
1     2017   1   2   46  194  2586.3  2.27 -2.24  8.13  84.16  ...  <NA>   
2     2017   1   3   46  194  2586.3  1.92 -1.84  6.88  83.48  ...  <NA>   
3     2017   1   4   46  194  2586.3  2.82 -2.61  9.56  79.68  ...  <NA>   
4     2017   1   5   46  194  2586.3  3.01 -3.44  7.99  51.05  ...  <NA>   
...    ...  ..  ..  ...  ...     ...   ...   ...   ...    ...  ...   ...   
1456  2020  12  27   46  194  2586.3  1.83 -0.08  4.24  60.21  ...  1.25   
1457  2020  12  28   46  194  2586.3  2.22 -0.17  3.23  77.08  ...  0.84   
1458  2020  12  29   46  194  2586.3  1.34 -2.66  3.25  73.42  ...  <NA>   
1459  2020  12  30   46  194  2586.3  2.93  0.61  5.29  75.44  ...  <NA>   
1460  2020  12  31   46  194  2586.3  0.94 -3.58  3.04  65.92  ...  <NA>   

        125  ug.m-3   24h     ---                  >TS       ---         
     Median StDev.5 Max.2 COMx.11 Min.11 COMn.11  GP.9 A[km2].10 A[%].9  
0      <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
1      <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
2      <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
3      <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
4      <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
...     ...     ...   ...     ...    ...     ...   ...       ...    ...  
1456   1.24     0.1  1.78    3794   0.77    3809     0        0.     0.  
1457   0.84    0.09  1.37    3794   0.51    3809     0        0.     0.  
1458   <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
1459   <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  
1460   <NA>    <NA>  <NA>    <NA>   <NA>    <NA>  <NA>      <NA>   <NA>  

[1461 rows x 123 columns]